<a href="https://colab.research.google.com/github/Areeff10/resume_screening/blob/main/resume_screeing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import regex as re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
data=pd.read_csv('/content/UpdatedResumeDataSet.csv')
data.head(5)

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [ ]:
data['Resume'][144]

'IT SKILLS Languages: C (Basic), JAVA (Basic) Web Technologies: HTML5, CSS3, Bootstrap, JavaScript, jQuery, Corel Draw, Photoshop, Illustrator Databases: MySQL5.0 IDE & Tools: Sublime Text, Notepad Operating Systems: Windows XP, Windows 7Education Details \r\nSeptember 2015 Bachelor of Engineer Information technology Nagpur, Maharashtra Nagpur University\r\nMay 2011 HSC Secondary & Higher Secondary  State Board of Secondary\r\nJune 2009 SSC Secondary & Higher Secondary  Maharashtra State Board of Secondary\r\nWeb and Graphics Designer \r\n\r\nWeb and Graphics Designer - Virtuous Media Point, Pune\r\nSkill Details \r\nBOOTSTRAP- Exprience - 24 months\r\nHTML5- Exprience - 24 months\r\nJAVASCRIPT- Exprience - 24 months\r\njQuery- Exprience - 24 months\r\nCOREL DRAW- Exprience - 24 months\r\nAdobe Photoshop- Exprience - 24 months\r\nAdobe Illustrator- Exprience - 12 months\r\nCSS3- Exprience - 24 monthsCompany Details \r\ncompany - Virtuous Media Point\r\ndescription - \r\ncompany - CNC W

In [ ]:
data.shape

(962, 2)

In [ ]:
data.isnull().sum()

,0
Category,0
Resume,0


In [ ]:
data['Category'].value_counts()

,count
Category,
Java Developer,84
Testing,70
DevOps Engineer,55
Python Developer,48
Web Designing,45
HR,44
Hadoop,42
Blockchain,40
ETL Developer,40


## Balancing Dataset

In [ ]:
x=data.drop('Category', axis=1)
y=data['Category']

ros=RandomOverSampler(random_state=42)
x_sample,y_sample=ros.fit_resample(x,y)

balanced_df=pd.concat([x_sample,y_sample],axis=1)

print(balanced_df['Category'].value_counts())

Category
Data Science                 84
Electrical Engineering       84
Blockchain                   84
DotNet Developer             84
ETL Developer                84
Hadoop                       84
Database                     84
PMO                          84
Network Security Engineer    84
DevOps Engineer              84
Python Developer             84
Operations Manager           84
Automation Testing           84
HR                           84
SAP Developer                84
Business Analyst             84
Java Developer               84
Civil Engineer               84
Health and fitness           84
Sales                        84
Mechanical Engineer          84
Web Designing                84
Arts                         84
Advocate                     84
Testing                      84
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [ ]:
le=LabelEncoder()
le.fit(balanced_df['Category'])
balanced_df['Category']=le.transform(balanced_df['Category'])

## Cleaning dataset

In [ ]:
def cleanResume(txt):
    cleanText = re.sub('http\S+\s', ' ', txt)  #removes url
    cleanText = re.sub('RT|cc', ' ', cleanText)  # removes retweet and Carnon copy
    cleanText = re.sub('#\S+\s', ' ', cleanText)  #removes word after hastag
    cleanText = re.sub('@\S+', '  ', cleanText)  #removes word after at the rate
    cleanText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)  #remove all puntuation symbol
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText)   #remove all speical character
    cleanText = re.sub('\s+', ' ', cleanText)  #remove extra white spaces
    return cleanText


In [ ]:
balanced_df['Resume']=balanced_df['Resume'].apply(cleanResume)

## Vectorization

In [ ]:
tfidf=TfidfVectorizer(stop_words='english')
tfidf.fit(balanced_df['Resume'])
req_result=tfidf.transform(balanced_df['Resume'])

## Spliting dataset

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(req_result,balanced_df['Category'],test_size=0.2,random_state=42)

## RandomForest

In [ ]:
rf_classifier=RandomForestClassifier(n_estimators=10)
rf_classifier.fit(x_train,y_train)

RandomForestClassifier(n_estimators=10)

In [ ]:
preds =rf_classifier.predict(x_test)

In [ ]:
accuracy_score(y_test,preds)

0.9976190476190476

## KNeighbors

In [ ]:
knn=KNeighborsClassifier(n_neighbors=10)
knn.fit(x_train,y_train)
knn.score(x_test,y_test)

0.9952380952380953

In [ ]:
import pickle
pickle.dump(tfidf,open('tfidf.pkl','wb'))
pickle.dump(rf_classifier, open('clf.pkl', 'wb'))
pickle.dump(le, open("encoder.pkl",'wb'))

## Prediction System

In [ ]:
def system(input_resume):
    cleaned_txt = cleanResume(input_resume)

    tfidf_txt = tfidf.transform([cleaned_txt])

    predict = rf_classifier.predict(tfidf_txt)

    predicted_category_name = le.inverse_transform(predict)

    return predicted_category_name[0]


### Prediction-1

In [ ]:
system('''Skills * Programming Languages: Python (pandas, numpy, scipy, scikit-learn, matplotlib), Sql, Java, JavaScript/JQuery. * Machine learning: Regression, SVM, NaÃ¯ve Bayes, KNN, Random Forest, Decision Trees, Boosting techniques, Cluster Analysis, Word Embedding, Sentiment Analysis, Natural Language processing, Dimensionality reduction, Topic Modelling (LDA, NMF), PCA & Neural Nets. * Database Visualizations: Mysql, SqlServer, Cassandra, Hbase, ElasticSearch D3.js, DC.js, Plotly, kibana, matplotlib, ggplot, Tableau. * Others: Regular Expression, HTML, CSS, Angular 6, Logstash, Kafka, Python Flask, Git, Docker, computer vision - Open CV and understanding of Deep learning.Education Details

Data Science Assurance Associate

Data Science Assurance Associate - Ernst & Young LLP
Skill Details
JAVASCRIPT- Exprience - 24 months
jQuery- Exprience - 24 months
Python- Exprience - 24 monthsCompany Details
company - Ernst & Young LLP
description - Fraud Investigations and Dispute Services   Assurance
TECHNOLOGY ASSISTED REVIEW
TAR (Technology Assisted Review) assists in accelerating the review process and run analytics and generate reports.
* Core member of a team helped in developing automated review platform tool from scratch for assisting E discovery domain, this tool implements predictive coding and topic modelling by automating reviews, resulting in reduced labor costs and time spent during the lawyers review.
* Understand the end to end flow of the solution, doing research and development for classification models, predictive analysis and mining of the information present in text data. Worked on analyzing the outputs and precision monitoring for the entire tool.
* TAR assists in predictive coding, topic modelling from the evidence by following EY standards. Developed the classifier models in order to identify "red flags" and fraud-related issues.

Tools & Technologies: Python, scikit-learn, tfidf, word2vec, doc2vec, cosine similarity, NaÃ¯ve Bayes, LDA, NMF for topic modelling, Vader and text blob for sentiment analysis. Matplot lib, Tableau dashboard for reporting.

MULTIPLE DATA SCIENCE AND ANALYTIC PROJECTS (USA CLIENTS)
TEXT ANALYTICS - MOTOR VEHICLE CUSTOMER REVIEW DATA * Received customer feedback survey data for past one year. Performed sentiment (Positive, Negative & Neutral) and time series analysis on customer comments across all 4 categories.
* Created heat map of terms by survey category based on frequency of words * Extracted Positive and Negative words across all the Survey categories and plotted Word cloud.
* Created customized tableau dashboards for effective reporting and visualizations.
CHATBOT * Developed a user friendly chatbot for one of our Products which handle simple questions about hours of operation, reservation options and so on.
* This chat bot serves entire product related questions. Giving overview of tool via QA platform and also give recommendation responses so that user question to build chain of relevant answer.
* This too has intelligence to build the pipeline of questions as per user requirement and asks the relevant /recommended questions.

Tools & Technologies: Python, Natural language processing, NLTK, spacy, topic modelling, Sentiment analysis, Word Embedding, scikit-learn, JavaScript/JQuery, SqlServer

INFORMATION GOVERNANCE
Organizations to make informed decisions about all of the information they store. The integrated Information Governance portfolio synthesizes intelligence across unstructured data sources and facilitates action to ensure organizations are best positioned to counter information risk.
* Scan data from multiple sources of formats and parse different file formats, extract Meta data information, push results for indexing elastic search and created customized, interactive dashboards using kibana.
* Preforming ROT Analysis on the data which give information of data which helps identify content that is either Redundant, Outdated, or Trivial.
* Preforming full-text search analysis on elastic search with predefined methods which can tag as (PII) personally identifiable information (social security numbers, addresses, names, etc.) which frequently targeted during cyber-attacks.
Tools & Technologies: Python, Flask, Elastic Search, Kibana

FRAUD ANALYTIC PLATFORM
Fraud Analytics and investigative platform to review all red flag cases.
â¢ FAP is a Fraud Analytics and investigative platform with inbuilt case manager and suite of Analytics for various ERP systems.
* It can be used by clients to interrogate their Accounting systems for identifying the anomalies which can be indicators of fraud by running advanced analytics
Tools & Technologies: HTML, JavaScript, SqlServer, JQuery, CSS, Bootstrap, Node.js, D3.js, DC.js''')

'Data Science'

### Prediction-1

In [ ]:
system('''IT SKILLS Languages: C (Basic), JAVA (Basic) Web Technologies: HTML5, CSS3, Bootstrap, JavaScript, jQuery, Corel Draw, Photoshop, Illustrator Databases: MySQL5.0 IDE & Tools: Sublime Text, Notepad Operating Systems: Windows XP, Windows 7Education Details
September 2015 Bachelor of Engineer Information technology Nagpur, Maharashtra Nagpur University
May 2011 HSC Secondary & Higher Secondary  State Board of Secondary
June 2009 SSC Secondary & Higher Secondary  Maharashtra State Board of Secondary
Web and Graphics Designer

Web and Graphics Designer - Virtuous Media Point, Pune
Skill Details
BOOTSTRAP- Exprience - 24 months
HTML5- Exprience - 24 months
JAVASCRIPT- Exprience - 24 months
jQuery- Exprience - 24 months
COREL DRAW- Exprience - 24 months
Adobe Photoshop- Exprience - 24 months
Adobe Illustrator- Exprience - 12 months
CSS3- Exprience - 24 monthsCompany Details
company - Virtuous Media Point
description -
company - CNC Web World
description - Internship Program: At e-sense IT Solution pvt.ltd. Nagpur as a Web Designing and Developement.
* Presented in Project Competition in Innovesta 15 of Priyadarshini Indira Gandhi College of Engineering, Nagpur.
* Presented in National Level Paper Presentation in TECH-WAVE 2015 of S.R.M.C.E., Nagpur.
company - e-sense IT Solution pvt.ltd
description - Key Result Areas:
* Designed websites solutions by studying information needs, conferring with users, and studying systems flow, data usage, and work processes.
* Understood process requirements and provided use cases for business, functional & technical requirements.
* Interacted with users for requirement gathering, prepared functional specifications and low-level design documents.
* Participated in the Software Development Life cycle (SDLC) and Agile methodology right from requirement analysis,
* Performed detailed design of modules along with their implementation, and documentation integrated software modules
Developed by other team members.

Highlights:
* Developed various modules as per customer requirement and identified and fixed number of bugs related to code, Database connectivity, UI Defects and so on.
* Analyzed and modified existing codes to incorporate a number of changes in the application / user requirements, wrote new codes as required.
* Coded, implemented and integrated complex programs using technologies such as HTML5, CSS3, JavaScript, jQuery, bootstrap.
* Having good command on Graphics designing with effective ideas.

PROJECTS

* www.nitka.com, Nagpur united corporation (admin), Mintmetrix.com, Tagline videos (admin), Smartbadge (admin): -
In all projects I have used technologies like HTML5, CSS3, Bootstrap, JavaScript, jQuery and text editor as sublime text.

* www.shreekiaspack.co.in, www.3staragroproducts.com, www.luckystationery.co.in: - used technologies like HTML5, CSS3,
Bootstrap, javascript and text editor as notepad++.

* Design various Logos, Brochures, Advertising Banners, Visiting Cards, Pamphlet, Hoardings etc.

B.E. FINAL YEAR PROJECT

* Major Project: -

Title: WEB BASED DISEASE DIAGNOSIS EXPERT SYSTEM.
Duration: 1 Year

Description: In this project we provide a website in which doctor gives online consultation for particular disease. System gives better suggestions for any health problems.

* Mini Project Development-

* SHOPPING MANAGEMENT SYSTEM Developed in C++.

CURRICULUM & EXTRA CURRICULUM  ACTIVITIES
company - FACE-IT
description - Co-ordinator in project competition.''')

'Web Designing'